# OBJECTIVE:Deal with activities which overlap w.r.t timestamps in report

In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import time

In [21]:
smart_glasses=pd.read_csv("glasses.csv")
smart_watch=pd.read_csv("smartwatch.csv")
smart_phone=pd.read_csv("smartphone.csv")
report=pd.read_csv("report.csv")

In [22]:
smart_glasses=smart_glasses.rename(columns={'DATE':'TIMESTAMP'})
smart_watch=smart_watch.rename(columns={'timestamp':'TIMESTAMP'})
smart_phone=smart_phone.rename(columns={'timestamp':'TIMESTAMP'})
report=report.rename(columns={'from':'FROM','to':'TO'})

In [23]:
smart_phone.TIMESTAMP =pd.to_datetime(smart_phone.TIMESTAMP)
smart_glasses.TIMESTAMP=pd.to_datetime(smart_glasses.TIMESTAMP)
smart_watch.TIMESTAMP=pd.to_datetime(smart_watch.TIMESTAMP)
report.FROM=pd.to_datetime(report.FROM)
report.TO=pd.to_datetime(report.TO)

In [24]:
l1=[]
for i in range(0,len(report)):
    l1.append([report.loc[i,'FROM'],report.loc[i,'TO']])
# l1=pdf1['from'].tolist()
s1=pd.Series(l1)
report['FROM_TO']=s1
report.head(3)

,index,activity_type,duration,FROM,TO,comment,FROM_TO
0,0,Video games,01:43,2017-07-01 21:34:00,2017-07-01 23:17:00,NaN,"[2017-07-01 21:34:00, 2017-07-01 23:17:00]"
1,1,In computer,00:03,2017-07-01 21:29:00,2017-07-01 21:32:00,NaN,"[2017-07-01 21:29:00, 2017-07-01 21:32:00]"
2,2,At home,13:35,2017-07-01 21:13:00,2017-07-02 10:49:00,NaN,"[2017-07-01 21:13:00, 2017-07-02 10:49:00]"


In [6]:
print("\n       SMARTGLASS\n\n")
print(smart_glasses.head(2))
print("\n       SMARTWATCH\n\n")
print(smart_watch.head(2))
print("\n       SMARTPHONE\n\n")
print(smart_watch.head(2))
print("\n       REPORT\n\n")
print(report.head(2))


       SMARTGLASS


   NUM               TIMESTAMP  ACC_X  ACC_Y  ACC_Z  GYRO_X  GYRO_Y  GYRO_Z  \
0    1 2017-06-29 08:00:00.406  -3242  10396  -6489    9117    2018    -819   
1    3 2017-06-29 08:00:00.426  -4014  11348 -15131    3359    -882   -2009   

   EOG_L  EOG_R  EOG_H  EOG_V  
0   -330   -186   -144    258  
1   -305   -156   -149    230  

       SMARTWATCH


    index         source               TIMESTAMP  \
0  164349        battery 2017-06-29 07:45:56.262   
1  164350  accelerometer 2017-06-29 07:46:01.263   

                                     values  
0                                    ['98']  
1  ['-0.3440857', '0.68330383', '9.967361']  

       SMARTPHONE


    index         source               TIMESTAMP  \
0  164349        battery 2017-06-29 07:45:56.262   
1  164350  accelerometer 2017-06-29 07:46:01.263   

                                     values  
0                                    ['98']  
1  ['-0.3440857', '0.68330383', '9.967361']  

       REPOR

In [7]:
print("\n       SMARTGLASS\n\n")
print(smart_glasses.dtypes)
print("\n       SMARTWATCH\n\n")
print(smart_watch.dtypes)
print("\n       SMARTPHONE\n\n")
print(smart_phone.dtypes)
print("\n       REPORT\n\n")
print(report.dtypes)


       SMARTGLASS


NUM                   int64
TIMESTAMP    datetime64[ns]
ACC_X                 int64
ACC_Y                 int64
ACC_Z                 int64
GYRO_X                int64
GYRO_Y                int64
GYRO_Z                int64
EOG_L                 int64
EOG_R                 int64
EOG_H                 int64
EOG_V                 int64
dtype: object

       SMARTWATCH


index                 int64
source               object
TIMESTAMP    datetime64[ns]
values               object
dtype: object

       SMARTPHONE


index                 int64
source               object
TIMESTAMP    datetime64[ns]
values               object
dtype: object

       REPORT


index                     int64
activity_type            object
duration                 object
FROM             datetime64[ns]
TO               datetime64[ns]
comment                  object
FROM_TO                  object
dtype: object


In [8]:
report['activity_type'].value_counts()

Walk                                    29
In bus                                  19
In computer                             18
Eat                                     18
At home                                 11
Train                                    9
In vehicle                               8
Video games                              6
Meeting                                  5
Cooking                                  4
Pause                                    4
Shop                                     3
Movie                                    3
Walking&party                            2
Work                                     2
Running                                  2
Took off glasses                         1
Phone was out of the pocket (forgot)     1
Picnic                                   1
On bus stop                              1
Football                                 1
Shopping& wearing                        1
Watching TV                              1
Sleep      

# CREATING TIMESTAMPS FOR FINDING OUT ACTIVITIES CORRESPONDING TO THEM


In [25]:
l1=[]

def time_append(arg):
    start_time=arg[0]
    end_time=arg[1]
    temp_time=pd.date_range(start=start_time,end=end_time,freq='1min')
    #THE TIME GAP BETWEEN EACH ROWS COULD BE ADJUSTED BY CHANGING THE VALUE OF 'freq' IN ABOVE FUNCTION
    for i in temp_time:
        l1.append(i)
    return(temp_time)
t=(report['FROM_TO'].map(time_append))
df={'FROM':pd.Series(l1)}

CHECK=pd.DataFrame(df)
CHECK['TO']=CHECK['FROM']+dt.timedelta(minutes=1)
CHECK['TIMESTAMP']=CHECK['FROM']+dt.timedelta(minutes=0.5)
#REMOVING DUPLICATE ROWS
CHECK.drop_duplicates(keep='first')
CHECK['FROM_TO'] = list(zip(CHECK['FROM'], CHECK['TO']))
CHECK.drop(['FROM','TO'],axis=1,inplace=True)


#REMOVING DUPLICATE ROWS
CHECK.drop_duplicates(keep='first')


print(CHECK.shape)
print(CHECK.head(3))

(8677, 2)
            TIMESTAMP                                     FROM_TO
0 2017-07-01 21:34:30  (2017-07-01 21:34:00, 2017-07-01 21:35:00)
1 2017-07-01 21:35:30  (2017-07-01 21:35:00, 2017-07-01 21:36:00)
2 2017-07-01 21:36:30  (2017-07-01 21:36:00, 2017-07-01 21:37:00)


# Finding out the activities across the respective timestamps

In [26]:
def checkf1(arg):
    report['check']=False
    report['check']=((report['FROM']<=arg)&(report['TO']>arg))
    a=report['activity_type'][report['check']]
    a.drop_duplicates(keep='first')
    a=a.tolist()
    if(len(a)>0):
        b=a[0]
        for i in range(1,len(a)):
            b=b+"/"+a[i]
        return(b)
    return("null")

    
CHECK['activity']=CHECK['TIMESTAMP'].map(checkf1)
report.drop('check',axis=1,inplace=True)
CHECK

,TIMESTAMP,FROM_TO,activity
0,2017-07-01 21:34:30,"(2017-07-01 21:34:00, 2017-07-01 21:35:00)",Video games/At home
1,2017-07-01 21:35:30,"(2017-07-01 21:35:00, 2017-07-01 21:36:00)",Video games/At home
2,2017-07-01 21:36:30,"(2017-07-01 21:36:00, 2017-07-01 21:37:00)",Video games/At home
3,2017-07-01 21:37:30,"(2017-07-01 21:37:00, 2017-07-01 21:38:00)",Video games/At home
4,2017-07-01 21:38:30,"(2017-07-01 21:38:00, 2017-07-01 21:39:00)",Video games/At home
5,2017-07-01 21:39:30,"(2017-07-01 21:39:00, 2017-07-01 21:40:00)",Video games/At home
6,2017-07-01 21:40:30,"(2017-07-01 21:40:00, 2017-07-01 21:41:00)",Video games/At home
7,2017-07-01 21:41:30,"(2017-07-01 21:41:00, 2017-07-01 21:42:00)",Video games/At home
8,2017-07-01 21:42:30,"(2017-07-01 21:42:00, 2017-07-01 21:43:00)",Video games/At home
9,2017-07-01 21:43:30,"(2017-07-01 21:43:00, 2017-07-01 21:44:00)",Video games/At home


In [27]:
CHECK.sort_values('TIMESTAMP',inplace=True)
CHECK.drop_duplicates()

,TIMESTAMP,FROM_TO,activity
7632,2017-06-29 09:57:30,"(2017-06-29 09:57:00, 2017-06-29 09:58:00)",Eat
7633,2017-06-29 09:58:30,"(2017-06-29 09:58:00, 2017-06-29 09:59:00)",Eat
7634,2017-06-29 09:59:30,"(2017-06-29 09:59:00, 2017-06-29 10:00:00)",Eat
7635,2017-06-29 10:00:30,"(2017-06-29 10:00:00, 2017-06-29 10:01:00)",Eat
7636,2017-06-29 10:01:30,"(2017-06-29 10:01:00, 2017-06-29 10:02:00)",Eat
7637,2017-06-29 10:02:30,"(2017-06-29 10:02:00, 2017-06-29 10:03:00)",Eat
7638,2017-06-29 10:03:30,"(2017-06-29 10:03:00, 2017-06-29 10:04:00)",Eat
7639,2017-06-29 10:04:30,"(2017-06-29 10:04:00, 2017-06-29 10:05:00)",Eat
7640,2017-06-29 10:05:30,"(2017-06-29 10:05:00, 2017-06-29 10:06:00)",Eat
7641,2017-06-29 10:06:30,"(2017-06-29 10:06:00, 2017-06-29 10:07:00)",Eat


In [12]:
# l=[]
# def append1(arg):
#     if(len(l)==0):
#         l.append(arg)
#         return(0)
#     else:
#         if(arg!=l[len(l)-1]):
#             l.append(arg)
#             return(1)
#         else:
#             return(2)
# CHECK['TEMP']=CHECK['activity'].map(append1)
# CHECK.drop(['TEMP','TIMESTAMP'],axis=1,inplace=True)
# print(len(l))
# l

267


['Eat',
 'null',
 'In vehicle',
 'null',
 'Walk',
 'Shop/Walk',
 'Took off glasses/Shop/Walk',
 'Shop/Walk',
 'Walk',
 'On bus stop/Walk',
 'On bus stop',
 'In bus',
 'null',
 'At home',
 'Eat/At home',
 'At home',
 'null',
 'In bus',
 'null',
 'Walking&party',
 'null',
 'In bus',
 'null',
 'Running',
 'null',
 'Eat',
 'null',
 'Walk',
 'Train',
 'Walk',
 'null',
 'Work',
 'In computer/Work',
 'Work',
 'In computer/Work',
 'Walk',
 'Shop',
 'Walk',
 'null',
 'Work',
 'null',
 'Walk',
 'null',
 'Train',
 'Walk',
 'At home/Eat',
 'At home',
 'null',
 'In bus',
 'Walk',
 'Meeting',
 'null',
 'Train',
 'null',
 'Walking&party',
 'null',
 'Sleep',
 'null',
 'Eat',
 'null',
 'In computer',
 'null',
 'Eat',
 'null',
 'In computer',
 'At home',
 'In computer/At home',
 'At home',
 'Video games/At home',
 'At home',
 'null',
 'In vehicle',
 'null',
 'In vehicle',
 'null',
 'Picnic ',
 'null',
 'In vehicle',
 'null',
 'At home',
 'In computer/At home',
 'At home',
 'In computer/At home',
 'At ho

In [29]:
CHECK['TEMP'] = list(zip(CHECK['FROM_TO'], CHECK['activity']))
CHECK.head()


,TIMESTAMP,FROM_TO,activity,TEMP
7632,2017-06-29 09:57:30,"(2017-06-29 09:57:00, 2017-06-29 09:58:00)",Eat,"((2017-06-29 09:57:00, 2017-06-29 09:58:00), Eat)"
7633,2017-06-29 09:58:30,"(2017-06-29 09:58:00, 2017-06-29 09:59:00)",Eat,"((2017-06-29 09:58:00, 2017-06-29 09:59:00), Eat)"
7634,2017-06-29 09:59:30,"(2017-06-29 09:59:00, 2017-06-29 10:00:00)",Eat,"((2017-06-29 09:59:00, 2017-06-29 10:00:00), Eat)"
7635,2017-06-29 10:00:30,"(2017-06-29 10:00:00, 2017-06-29 10:01:00)",Eat,"((2017-06-29 10:00:00, 2017-06-29 10:01:00), Eat)"
7636,2017-06-29 10:01:30,"(2017-06-29 10:01:00, 2017-06-29 10:02:00)",Eat,"((2017-06-29 10:01:00, 2017-06-29 10:02:00), Eat)"


# Making it into the report format

In [30]:
F=[]
T=[]
A=[]

def append2(arg):
    if(len(A)>0):
        if(arg[1]!=A[len(A)-1]):
            F.append(arg[0][0])
            A.append(arg[1])
            T.append(arg[0][1])
            return(1)
        else:
            T[len(T)-1]=arg[0][1]
            return(2)
    else:
        F.append(arg[0][0])
        A.append(arg[1])
        T.append(arg[0][1])
        return(0)
CHECK['TEMP1']=CHECK['TEMP'].map(append2)
CHECK.drop(['TEMP1'],axis=1,inplace=True)
report.drop('FROM_TO',axis=1,inplace=True)

print(len(A),len(F),len(T))


267 267 267


In [42]:
def splitf(arg):
    a=arg.split(" ")[2]
    return(a[3:8])

df={'FROM':pd.Series(F),'TO':pd.Series(T),'activity':pd.Series(A)}
final_report=pd.DataFrame(df)
final_report=final_report[final_report['activity']!='null']
final_report['DURATION']=final_report['TO']-final_report['FROM']

final_report['DURATION']=final_report['DURATION'].astype(str)
final_report['DURATION']=final_report['DURATION'].map(splitf)


# ..split(" ")[2])
final_report.to_csv("final_report.csv",sep=",")
final_report


,FROM,TO,activity,DURATION
0,2017-06-29 09:57:00,2017-06-29 10:17:00,Eat,20:00
2,2017-06-29 10:46:00,2017-06-29 11:07:00,In vehicle,21:00
4,2017-06-29 11:09:00,2017-06-29 12:12:00,Walk,03:00
5,2017-06-29 12:12:00,2017-06-29 12:21:00,Shop/Walk,09:00
6,2017-06-29 12:21:00,2017-06-29 12:29:00,Took off glasses/Shop/Walk,08:00
7,2017-06-29 12:29:00,2017-06-29 12:40:00,Shop/Walk,11:00
8,2017-06-29 12:40:00,2017-06-29 12:44:00,Walk,04:00
9,2017-06-29 12:44:00,2017-06-29 13:01:00,On bus stop/Walk,17:00
10,2017-06-29 13:01:00,2017-06-29 13:02:00,On bus stop,01:00
11,2017-06-29 13:02:00,2017-06-29 13:24:00,In bus,22:00


In [1]:
final_report.activity.value_counts()

NameError: name 'final_report' is not defined

In [ ]:
len(final_report)